In [ ]:
import GSL: hypergeom, sf_gamma

mybesselj(α, x) = (x / 2)^α / sf_gamma(α + 1) * hypergeom([], α + 1, -x^2 / 4)
function sphericalbesselj_1(n, x::AbstractFloat)
    if iszero(x)
        return iszero(n) ? one(x) : zero(x)
    else
        return √(π / 2x) * mybesselj(n + 0.5, x)
    end
end


In [ ]:
import SpecialFunctions
sphericalbesselj_2(ν, x) = √(π/2x)*SpecialFunctions.besselj(ν+1/2, x)

In [ ]:
function sphericalbesselj_3(n::Integer, x::T) where T<:AbstractFloat
    isnan(x) && return x
    n < 0 && throw(DomainError("n must be non-negative"))
    isinf(x) && return zero(T)
    iszero(x) && return iszero(n) ? one(T) : zero(T)

    if n > 0 && n ≥ x
        return T(SpecialFunctions.sphericalbesselj(n, x))
    end

    s0 = sin(x) / x
    iszero(n) && return s0

    s1 = (s0 - cos(x)) / x
    isone(n) && return s1

    sn = zero(T)
    for idx in 2:n
        sn = (2idx - 1) * s1 / x - s0
        s0, s1 = s1, sn
        # Overflow occurred already: terminate recurrence.
        isnan(sn) && return sn
    end
    return sn
end

In [ ]:
@code_warntype sphericalbesselj_3(29, 5.6)

In [ ]:
using PyCall
scipy_special = pyimport("scipy.special")

In [ ]:
n = 100
x = 10000.0
# Mathematica: -0.000072814706186135590960089481513765273217282769121987832010967...
println(sphericalbesselj_1(n, x))
println(sphericalbesselj_2(n, x))
println(sphericalbesselj_3(n, x))
println(scipy_special.spherical_jn(n, x))

In [ ]:
n = 10
x = 10000.0
# Mathematica: 0.0000310846680541186048422278797019184103613015346166689343934711
println(sphericalbesselj_1(n, x))
println(sphericalbesselj_2(n, x))
println(sphericalbesselj_3(n, x))
println(scipy_special.spherical_jn(n, x))

In [ ]:
n = 11
x = 2e9
# Mathematica: 2.0205491360122228731362952404647241160170695797177388073166 × 10^-10
println(sphericalbesselj_1(n, x))
# println(sphericalbesselj_2(n, x))
println(sphericalbesselj_3(n, x))
println(scipy_special.spherical_jn(n, x))

In [ ]:
n = 100
x = 10.0
# Mathematica: 5.83204018200587674682241088046*10^-90
println(sphericalbesselj_1(n, x))
println(sphericalbesselj_2(n, x))
println(sphericalbesselj_3(n, x))
println(scipy_special.spherical_jn(n, x))